# Section 6

This file runs the partial dependence plot (PDP) analysis for the top performing models using a causal interpretation of this data visualization method. Given the weighted data, we modify scikit-learn's PDP to include weights. We also calculate the change in PDP.

The PDP requires a trained model and data. Here, we use the hyper-parameters of the top model for each model-type (Logit and LGBM) and re-train it on the entire dataset. Then we use this model for the PDP.

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import time

#from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence
from matplotlib import pyplot as plt

# local modules
from _utils import *
from _partial_dependence_weighted import partial_dependence

In [2]:
# general settings  
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=16)
plt.rc('legend', fontsize=16)
plt.rc('lines', linewidth=2)
plt.rc('axes', linewidth=2)
plt.rc('axes', edgecolor='k')
plt.rc('xtick.major', width=2)
plt.rc('xtick.major', size=10)
plt.rc('ytick.major', width=2)
plt.rc('ytick.major', size=10)
plt.rc('pdf', fonttype=42)
plt.rc('ps', fonttype=42)

In [3]:
# working directory
os.chdir("..")
wd = os.getcwd()
# data folder
data_path = wd + '/' + 'data' + '/'
# results folder
resu_path = wd + '/' + 'results' + '/'

In [4]:
# use 'w' for weighted or 'u' for unweighted - the latter used as default
experiment_type = 'u' 

if experiment_type == 'w':
    filename = 'experiment_results_w.pkl'
    print('---> run WEIGHTED')
else:
    filename = 'experiment_results.pkl'
    print('---> run UNWEIGHTED')

# we use the results from Section 4
with open(resu_path + filename, 'rb') as f: 
    [target, 
     categorical_columns, 
     cat_feats, con_feats, ord_feats, all_feats, 
     state_encoder, sex_encoder, race_encoder, 
     weight_column, 
     dataset, 
     experiment_results, final_results] = pickle.load(f)

---> run UNWEIGHTED


In [5]:
dataset.head(5)

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,Y,STATE
0,32.0,2.0,19.0,5.0,5510.0,2.0,7.0,40.0,0,6,1,0
1,61.0,4.0,16.0,1.0,4220.0,2.0,1.0,40.0,1,4,0,0
2,65.0,2.0,21.0,1.0,6200.0,2.0,0.0,35.0,1,3,1,0
3,38.0,1.0,17.0,4.0,310.0,46.0,0.0,30.0,0,6,0,0
4,50.0,1.0,21.0,1.0,3255.0,328.0,0.0,50.0,0,1,1,0


In [9]:
weight_column is None

True

## Train a single model using the top best-hyparams and on the full dataset

In [10]:
# store results for both models
trained_clfs = {}

# param 1
dataset_type = 'bible_belt'

# param 2
for clf_type in ['LR','LGBM2']:
    
#     # get dataset
#     dataset = datasets[dataset_type].copy()
#     print(dataset.shape)

    # get best model params
    temp_experiment_results = experiment_results[
        (experiment_results['clf_name'] == clf_type) & (experiment_results['dataset'] == dataset_type)
        ]
    best_hyparams = temp_experiment_results.iloc[0]['best_hyparams']
    del temp_experiment_results
    
    if weight_column is not None:
        # match weights
        print(weight_column.shape)
        weight_column = weight_column.filter(items=dataset.index)
        print(weight_column.shape)
    else:
        print('no weights provided')
    
    # divide data accordingly
    X = dataset.copy()
    y = dataset[target]
    X.drop(columns = [target], inplace = True)
    
    # train on the full data
    clf = build_model(clf_type, categorical_columns, best_hyparams)    
    clf.fit(X, y, clf__sample_weight = weight_column)
    
    trained_clfs[clf_type] = clf
    
print('DONE')

no weights provided


D:\Programs\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


no weights provided
[LightGBM] [Warning] feature_fraction is set=0.7789355153603348, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7789355153603348
[LightGBM] [Warning] lambda_l1 is set=0.6437596878291109, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6437596878291109
[LightGBM] [Warning] bagging_fraction is set=0.8577577792661564, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8577577792661564
[LightGBM] [Warning] lambda_l2 is set=0.40648792269514766, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.40648792269514766
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
DONE


In [11]:
trained_clfs.keys()

dict_keys(['LR', 'LGBM2'])

## Beta coefficients for LR

In [13]:
temp_betas_0 = pd.DataFrame(zip(trained_clfs['LR']['one hot'].get_feature_names(), 
                                np.transpose(trained_clfs['LR']['clf'].coef_)), 
                            columns=['features', 'betas'])
# remove [] and round up
temp_betas_0['betas'] = temp_betas_0['betas'].map(lambda x: round(x[0], 5))
# Keep track of these for later
temp_betas_0

D:\Programs\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,features,betas
0,onehot__x0_0,-2.25113
1,onehot__x0_1,-1.51440
2,onehot__x1_0,-0.39583
3,onehot__x1_1,-0.67840
4,onehot__x1_2,-0.37106
...,...,...
502,COW,-0.28982
503,SCHL,4.23293
504,POBP,-0.34394
505,RELP,-1.26630


## Run (weighted) PDP

In the paper, we provided country-specific weights to address potential sample selection bias in the survey data. We then explored the causal importance of each *Theme* in the survey. Under this version with the US Bible Belt states, we are not considering state-specific weights. The pipeline below applies for the weighted version, which we explore in future work.

In [16]:
columns_of_interest = ['MAR', 'SCHL']

In [19]:
# store results for both models
pdp_res = {}

delta_pdp = pd.DataFrame(columns=['Theme', 'LR', 'LG']) # here, delta = 0 - 10 

for theme_var in columns_of_interest:
    print(theme_var)
    
    if weight_column is not None:
        print('runing standard PDP')
    else:
        print('running weighted PDP')
    
    # to get the full range of X vals:
    grid_res = len(X[theme_var].unique()) + 1
    
    # calculate PD for both classifiers
    pd_lg = partial_dependence(trained_clfs['LGBM2'], X, [theme_var],
                               grid_resolution=grid_res,
                               method='brute', kind='average', response_method='predict_proba', 
                               sample_weight=weight_column)
    pd_lr = partial_dependence(trained_clfs['LR'], X, [theme_var],
                               grid_resolution=grid_res,
                               method='brute', kind='average', response_method='predict_proba', 
                               sample_weight=weight_column)
    # store results
    pdp_res[theme_var] = {}
    pdp_res[theme_var]['LGBM2'] = pd_lg
    pdp_res[theme_var]['LR'] = pd_lr
    
    # store deltas
    temp_delta_pdp = {}
    temp_delta_pdp['Theme'] = theme_var.split('_')[0]
    temp_delta_pdp['LR'] = round(pd_lr['average'][0][0] - pd_lr['average'][0][-1], 3)
    temp_delta_pdp['LG'] = round(pd_lg['average'][0][0] - pd_lg['average'][0][-1], 3)
    delta_pdp = delta_pdp.append(temp_delta_pdp, ignore_index=True)
    del temp_delta_pdp
    
    # create and save plot
    plt.plot(pd_lg['values'][0], pd_lg['average'][0], label='LGBM')
    plt.plot(pd_lr['values'][0], pd_lr['average'][0], label='LR')
    
    plt.legend()
    plt.xlabel(theme_var.split('_')[0] + ' theme $t$')
    plt.ylabel('$\hat{b}_T (t)$')
    #plt.title('PDP for {} theme'.format(theme_var.split('_')[0]))
    plt.xlim([0, 10])
    #plt.ylim([0, np.max(pd_lg['average'] + pd_lr['average'])[0].round(1) + 0.25])
    plt.ylim([0, pd_lg['average'][0].max().round(1) + 0.25])

    # save
    plt.savefig(fname=resu_path + 'PDPs\\' + 'lar_pdp_{}_unweighted.pdf'.format(theme_var.split('_')[0]), 
                bbox_inches='tight', 
                dpi=400)
    
    # close with current plot
    plt.clf()
    
print('DONE')

MAR
running weighted PDP


C:\Users\jose\AppData\Local\Temp\3\ipykernel_15008\1386432967.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  delta_pdp = delta_pdp.append(temp_delta_pdp, ignore_index=True)
meta NOT subset; don't know how to subset; dropped


SCHL
running weighted PDP


C:\Users\jose\AppData\Local\Temp\3\ipykernel_15008\1386432967.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  delta_pdp = delta_pdp.append(temp_delta_pdp, ignore_index=True)
meta NOT subset; don't know how to subset; dropped


DONE


<Figure size 640x480 with 0 Axes>

In [20]:
delta_pdp.sort_values(by='LR', ascending=False, inplace=True)#.reset_index(drop=True, inplace=True)
delta_pdp

,Theme,LR,LG
0,MAR,0.064,0.037
1,SCHL,-0.404,-0.358


In [21]:
pdp_res.keys()

dict_keys(['MAR', 'SCHL'])